In [51]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘edgar_allan_poe.txt’ already there; not retrieving.

File ‘robert_frost.txt’ already there; not retrieving.



In [52]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [53]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [54]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    #remove null character in the end and convert all letters to lowercase.
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [55]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [56]:
len(Ytrain), len(Ytest)

(1615, 539)

In [57]:
idx = 1
word2idx = {'<unk>': 0}

In [58]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [59]:
word2idx

{'<unk>': 0,
 'he': 1,
 'steered': 2,
 'was': 3,
 'really': 4,
 'straight': 5,
 'away': 6,
 'make': 7,
 'yourself': 8,
 'up': 9,
 'a': 10,
 'cheering': 11,
 'song': 12,
 'of': 13,
 'how': 14,
 'alas': 15,
 'i': 16,
 'cannot': 17,
 'feel': 18,
 'for': 19,
 'tis': 20,
 'not': 21,
 'feeling': 22,
 'shes': 23,
 'in': 24,
 'earnest': 25,
 'it': 26,
 'appears': 27,
 'along': 28,
 'the': 29,
 'ramparts': 30,
 'plumed': 31,
 'and': 32,
 'pallid': 33,
 'thus': 34,
 'pacified': 35,
 'psyche': 36,
 'kissed': 37,
 'her': 38,
 'can': 39,
 'out': 40,
 'old': 41,
 'grandsir': 42,
 'stark': 43,
 'distinctly': 44,
 'something': 45,
 'must': 46,
 'have': 47,
 'learned': 48,
 'riding': 49,
 'trains': 50,
 'while': 51,
 'invested': 52,
 'him': 53,
 'with': 54,
 'reasons': 55,
 'sweet': 56,
 'hopes': 57,
 'thee': 58,
 'thine': 59,
 'looks': 60,
 'as': 61,
 'if': 62,
 'some': 63,
 'magic': 64,
 'sun': 65,
 'heres': 66,
 'hoping': 67,
 'she': 68,
 'gets': 69,
 'drink': 70,
 'safely': 71,
 'to': 72,
 'their':

In [60]:
len(word2idx)

2510

In [61]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [62]:
# initialize A and pi matrices - for both classes(Add one smoothing)
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [63]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [64]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [65]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [67]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3318885448916409, 0.6681114551083591)

In [68]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [69]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [70]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9956656346749226


In [71]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8144712430426716


In [72]:
from sklearn.metrics import confusion_matrix, f1_score
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 529,    7],
       [   0, 1079]])

In [73]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 90,  92],
       [  8, 349]])

In [74]:
f1_score(Ytrain, Ptrain)

0.9967667436489607

In [75]:
f1_score(Ytest, Ptest)

0.8746867167919798